In [ ]:
import gradio as gr
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# --------------------------
# Setup
# --------------------------

load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-nano")

if not OPENAI_KEY:
    raise RuntimeError("OPENAI_API_KEY is not set in the environment")

client = OpenAI(api_key=OPENAI_KEY)

class JobExtraction(BaseModel):
    role: str = Field(..., description="Job title or position name.")
    must_have: list[str] = Field(..., description="Top required skills or qualifications.")
    nice_to_have: list[str] = Field(default_factory=list, description="Optional but desirable skills.")
    responsibilities: str = Field(..., description="Key responsibilities of the role.")

/Users/vitorlaguna/repos/curriclick/ai/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def greet(user_prompt):

    system_prompt = (
        "Extract the role and skills from this job description. "
        "Return ONLY JSON that matches the schema."
        "\n\n<your job text here>"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    completion = client.chat.completions.parse(
        model=OPENAI_MODEL,
        messages=messages,
        response_format=JobExtraction,
    )

    return completion.choices[0].message

In [3]:
response = greet("""

Sobre a vaga
Na Stefanini, acreditamos no poder da colaboração. Co-criamos soluções inovadoras em parceria com nossos clientes, combinando tecnologia de ponta, inteligência artificial e a criatividade humana. Estamos na vanguarda da resolução de problemas de negócios, proporcionando impacto real em escala global.
 
Ao se juntar à Stefanini, você se torna parte de uma jornada global de transformação. Estamos empenhados em criar impacto positivo não apenas nos negócios, mas também na vida de nossos colaboradores. Se você procura uma oportunidade de crescimento profissional em uma empresa que valoriza inovação, respeito, autonomia e parceria, você encontra aqui! 



Junte-se a nós e seja parte da mudança!


Estamos em busca de um Desenvolvedor Full Stack para se juntar à nossa equipe. Este profissional será responsável por desenvolver e manter aplicações web, com foco em alta disponibilidade e baixa latência, além de garantir a qualidade e a automatização de testes.


Principais atividades:
- Desenvolvimento e manutenção de aplicações web;
- Implementação das funcionalidades levantadas pelo departamento de análise;
- Integração de sistemas;
- Automação de testes;
- Desenvolvimento de funcionalidades no Back-End;
- Desenvolvimento de funcionalidades no Front-End.


Requisitos:
- Experiência prévia nas linguagens informadas;
- Conhecimento avançado em React, Angular, Node.js, Typescript, PHP e AWS;
- Conhecimento em metodologias ágeis (Scrum/Kanban);
- Inglês Intermediário.


Diferenciais:
- Conhecimentos em Docker e Kubernetes;
- Experiência com TDD e BDD;
- Conhecimento em CI/CD.


Benefícios:


🍛 Vale Alimentação ou Vale Refeição;
👨🏼 🎓 Desconto em cursos, universidades e instituições de idiomas;
📚 Academia Stefanini - plataforma com cursos online, gratuitos, atualizados e com certificado;
🗣 Mentoring;
💉 Clube de vantagens para consultas e exames;
🏥 Assistência Médica;
🦷Assistência Odontológica;
🛫 Clube de viagens;
🐶 Convênio para Pet;
e muito mais...




""")

In [5]:
print(response.content)

{"role":"Desenvolvedor Full Stack","must_have":["React","Angular","Node.js","Typescript","PHP","AWS","Metodologias ágeis (Scrum/Kanban)","Inglês Intermediário"],"nice_to_have":["Docker","Kubernetes","TDD","BDD","CI/CD"],"responsibilities":"Desenvolvimento e manutenção de aplicações web; Implementação das funcionalidades levantadas pelo departamento de análise; Integração de sistemas; Automação de testes; Desenvolvimento de funcionalidades no Back-End; Desenvolvimento de funcionalidades no Front-End."}


## 🧠 Interview Agent

The **Interview Agent** is responsible for collecting and maintaining structured information about a user’s professional profile, ensuring that each interaction gradually enriches its understanding of the candidate.
It acts as the **first step** of the candidate evaluation process, providing a clean, structured summary that feeds directly into the **CVBuilder Agent**.

### 🎯 Core Functionality

When provided with a job description in JSON format — containing fields such as `role`, `must_have`, `nice_to_have`, and `responsibilities` — the Interview Agent:

1. **Analyzes required skills**
   It scans all `must_have` and `nice_to_have` skills listed in the job input.

2. **Consults the local database**
   The agent checks whether it already possesses stored information about the current user, including:

   * Technical skills and proficiency levels
   * Professional experience and key projects
   * Educational background
   * Language proficiency
   * Certifications
   * Contact details (email, phone number)
   * Location (state, city, neighborhood)

3. **Identifies missing information**
   For any skills or attributes not yet known, the agent formulates concise, formal questions in a single response.
   Example:

   ```
   What is your experience with Typescript?
   Have you ever worked with Kubernetes?
   What is your level of expertise with CI/CD?
   ```

4. **Updates the knowledge base**
   After the user provides answers, the agent automatically saves the new data in its local database, ensuring that each piece of information is permanently available for future interviews — eliminating repetitive questioning.

5. **Generates a Fit Summary**
   Once all relevant information is collected, the Interview Agent produces a structured summary detailing how the user’s skills, background, and experience align with the requirements of the given job description.
   This output serves as the input for the **CVBuilder Agent**, which uses it to automatically generate or update the candidate’s résumé.

---

### 🧩 Example Flow

**Input:**

```json
{
  "role": "Desenvolvedor Full Stack",
  "must_have": ["React", "Node.js", "Typescript", "AWS"],
  "nice_to_have": ["Docker", "CI/CD"]
}
```

**System Behavior:**

1. The agent checks which of these technologies it already knows the user has experience with.
2. It asks questions only about missing ones (e.g., Docker and CI/CD).
3. It stores the answers locally.
4. It outputs a **comprehensive fit report**, summarizing the candidate’s technical and professional match for the position.